In [ ]:
import pandas as pd
import sys
sys.path.append("../")
from cli import CreateDatasetSpecies
from glob import glob
FILE = "recon_microbe.tsv"
TRAIN_DB = "/hpc/group/singhlab/rawdata/afdbMert/clean70_0/clean70_db"
output = "/hpc/group/singhlab/rawdata/afdbMert/jay"


ids = [208964]

computed = set()


for i in ids:
    species = int(i)
    if species in computed:
        continue
    else:
        create_dataset = CreateDatasetSpecies(species, f"{output}/{species}", TRAIN_DB)
        create_dataset.run()


/hpc/group/singhlab/rawdata/afdbMert/jay/208964/208964_4f726a52-f94b-4a2b-80ee-d3287f5068fd


Copying gs://public-datasets-deepmind-alphafold-v4/proteomes/proteome-tax_id-208964-0_v4.tar...
/ [1/1 files][649.2 MiB/649.2 MiB] 100% Done                                    
Operation completed over 1 objects/649.2 MiB.                                    


/hpc/group/singhlab/rawdata/afdbMert/jay/208964/208964_4f726a52-f94b-4a2b-80ee-d3287f5068fd
running foldseek
foldseek easy-search --cov-mode 2 -e 0.1 /hpc/group/singhlab/rawdata/afdbMert/jay/208964/208964_4f726a52-f94b-4a2b-80ee-d3287f5068fd /hpc/group/singhlab/rawdata/afdbMert/clean70_0/clean70_db /hpc/group/singhlab/rawdata/afdbMert/jay/208964208964_aln.m8 /hpc/group/singhlab/user/me196/tmp/tmpu1bmsuzn/aln8213069190711167087


In [3]:
import pandas as pd
import sys
sys.path.append("../")
from cli import Predict
from glob import glob
FILE = "recon_microbe.tsv"
TRAIN_DB = "/hpc/group/singhlab/rawdata/afdbMert/clean70_0/clean70_db"
TRAIN_CSV = "/hpc/group/singhlab/rawdata/afdbMert/clean70_0/clean70_0_res.csv"
output = "/hpc/group/singhlab/rawdata/afdbMert/jay"
model = "../models/"
PEFT = "/hpc/group/singhlab/rawdata/afdbMert/clean70_0_model/"
ids = [208964]

for id in ids:
    data = f"{output}/{id}/{id}_res.csv"
    aln = glob(f"{output}/{id}*_aln.m8")[0]
    p = Predict(data, aln,model, PEFT,TRAIN_CSV)

    pred = p.run()
    pred.to_csv(f"{output}/{id}/{id}_pred.csv", index=False)
    print("Done with {}".format(id))



Map:   0%|          | 0/11264 [00:00<?, ? examples/s]

Done with 208964


In [4]:
ECS_OF_INTEREST = ["1.14.12.12", "1.14.15.3", "4.1.99.11", "1.14.13.243", "1.14.12.11", "1.14.15.26"]
from collections import defaultdict
results = defaultdict(lambda: defaultdict(list))
for id in ids:
    df = pd.read_csv(f"{output}/{id}/{id}_pred.csv")
    for ecs in ECS_OF_INTEREST:
        results[ecs]["id"].append(id)
        pred = df[df["EC"] == ecs]
        # extract Query Target and Bits
        pred = pred[["Query", "Target", "Bits"]]
        results[ecs]["pred"].append(pred)


In [5]:
with open("results.txt", "w") as f:
    for ecs in ECS_OF_INTEREST:
        f.write(f"Results for {ecs}\n\n")
        for i, id in enumerate(results[ecs]["id"]):
            print(f"Results for {id}")
            # If results[ecs]["pred"][i] is empty print No Match
            if results[ecs]["pred"][i].empty:
                print("No Match")
                continue
            else:
                f.write(f"\tOn Taxon Id {id}\n")
                f.write(results[ecs]["pred"][i].to_string())
                f.write("\n")


Results for 208964
No Match
Results for 208964
Results for 208964
No Match
Results for 208964
No Match
Results for 208964
No Match
Results for 208964
No Match
